# Compare models in a hybrid setup: Motion feedback- Force actuation

- numerical system: rotor aerodynamic simulator; calculate rotor loads
- physical turbine: measure motions and accelerations
- Ml model: multi-output regressor

In [1]:
from fowt_ml.pipeline import Pipeline

In [2]:
example_config_file = "../../src/example_config.yml"

In [3]:
my_pipeline = Pipeline(example_config_file)

In [4]:
# set correct path for mat file: add the correct path
my_pipeline.data_config["exp699"]["mat_file"] = "/home/sarah/temp/hybridlabs/data_example/exp699.mat"

In [5]:
# check the targets and predictors that are provided in the comfig
print("targets are:", my_pipeline.target_labels)
print("features are:", my_pipeline.predictors_labels)

targets are: ['force_aero_est6[0]', 'force_aero_est6[1]', 'force_aero_est6[2]', 'force_aero_est6[3]', 'force_aero_est6[4]', 'force_aero_est6[5]', 'force_tt_meas6[0]', 'force_tt_meas6[1]', 'force_tt_meas6[2]', 'force_tt_meas6[3]', 'force_tt_meas6[4]', 'force_tt_meas6[5]']
features are: ['acc_tb_meas3[0]', 'acc_tb_meas3[1]', 'acc_tb_meas3[2]', 'acc_tt_meas3[0]', 'acc_tt_meas3[1]', 'acc_tt_meas3[2]', 'pos_act6[0]', 'pos_act6[1]', 'pos_act6[2]', 'pos_act6[3]', 'pos_act6[4]', 'pos_act6[5]', 'spd_rot_act', 'wind_speed']


In [6]:
# check the models and metrics that are provided by the config
print(my_pipeline.model_names)
print(my_pipeline.metric_names)

{'ElasticNetRegression': {}, 'LeastAngleRegression': {}, 'LassoRegression': {}, 'LinearRegression': {}, 'SklearnGPRegressor': {'num_inducing': 100, 'num_latents': 3, 'num_epochs': 1}, 'RandomForest': {'n_estimators': 50, 'max_depth': 9, 'bootstrap': True, 'max_samples': 10000}, 'MultilayerPerceptron': {'hidden_layer_sizes': 10, 'max_iter': 10}}
['neg_mean_squared_error', 'neg_root_mean_squared_error', 'r2', 'model_fit_time']


In [7]:
# setup the pipeline
my_pipeline.setup(data="exp699")

In [8]:
# compare models
models, scores = my_pipeline.compare_models(sort="model_fit_time")

/home/sarah/miniconda3/envs/fowt_ml/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
2025/09/11 11:18:45 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'fowt-ml'}
INFO:fowt_ml.pipeline:Saving grid scores to grid_scores.csv
INFO:fowt_ml.pipeline:Saving best model to ONNX format in {file_name}


In [9]:
scores

,neg_mean_squared_error,neg_root_mean_squared_error,r2,model_fit_time
LeastAngleRegression,-7.127108,-2.259902,0.402951,0.22
LinearRegression,-7.122590,-2.259118,0.403461,0.22
MultilayerPerceptron,-7.265872,-2.313917,0.211441,8.55
LassoRegression,-7.762648,-2.360468,0.344400,11.21
ElasticNetRegression,-7.546190,-2.326244,0.361426,13.72
RandomForest,-7.001393,-2.225842,0.429356,17.62
SklearnGPRegressor,-26.811133,-3.971338,-0.411956,37.42


In [11]:
models

{'ElasticNetRegression': ElasticNet(),
 'LeastAngleRegression': Lars(),
 'LassoRegression': Lasso(),
 'LinearRegression': LinearRegression(),
 'SklearnGPRegressor': SklearnGPRegressor(num_epochs=1, num_inducing=100, num_latents=3),
 'RandomForest': RandomForestRegressor(max_depth=9, max_samples=10000, n_estimators=50),
 'MultilayerPerceptron': MLPRegressor(hidden_layer_sizes=10, max_iter=10)}